<a href="https://colab.research.google.com/github/bcarmoai/Colab/blob/main/Megaloop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import pandas
from sklearn.linear_model import LogisticRegression
os.chdir(r'U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\Bernard\segmentWOE')
from segmentWOE import segmentWOE_ML
os.chdir(r'U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\Bernard\dtreeAndROC')
from dtreeAndROC import dtreeRegROCWOE_ML
os.chdir(r'U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\Bernard\dtreeAndROC')

# =============================================================================
# Megaloop algorithm
# 1) Create baseline model for non-segmented dataset
# 2) For each variable in varlist that is not in ignore list
# 2.1) Find a value that splits population in half or creates a min 30% subset
# 2.2) Create dedicated models for each subset by Gini loop
# 2.3) Concatenate model scores from both subsets to calculate Gini improvement
# 2.4) Store Gini improvement and data required to plot ROC
# =============================================================================

def megaloop(train_to_split, valid_to_split, loop_var_list, train_var_list, segpath, segid, toignore_baseline=[], left_toignore=[],
             right_toignore=[], minsample = 200, split_min_group = 0.30, debug_mode=False, min_Gini_diff=0.003, max_vif=2.9):

# =============================================================================
# Code for testing
#     toignore_baseline=[]
#     left_toignore=[]
#     right_toignore=[]
#     minsample = 200
#     stratX_train_LimNonLim_ORI = pandas.read_csv(r"U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\4. Commercial model\BC_ConComm_LimNonLim_Retro_Aug_train_stratX_train.csv")
#     stratX_test_LimNonLim_ORI = pandas.read_csv(r"U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\4. Commercial model\BC_ConComm_LimNonLim_Retro_Aug_train_stratX_test.csv")
#     varlist_LimNonLim_ORI = ['RNILF04','CCJVAL24','GAZBKO36','EIOLF91','BSC423','LOCNCCJV','LSC251','SSC5','TSC135','INWORSTC','INTODFCU','IFOSF01']
#     train_to_split = (stratX_train_LimNonLim_ORI.copy())[['OpportunityID','target']+varlist_LimNonLim_ORI]
#     valid_to_split = stratX_test_LimNonLim_ORI.copy()[['OpportunityID','target']+varlist_LimNonLim_ORI]
#     loop_var_list = varlist_LimNonLim_ORI
#     loop_var_list = ['RNILF04']
#     train_var_list = loop_var_list
#     segpath=r"U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\4. Commercial model\megaloop";
#     segid="TESTRUN"
#     var = 'RNILF04'
# =============================================================================


    var_gini_gain = []

    if debug_mode:
        print('DEBUG: 001 Baseline')

    # 1) Create baseline model for non-segmented dataset
    # Replace raw values with WOE
    train_to_split_woe, valid_to_split_woe = segmentWOE_ML(segid, train_to_split, valid_to_split, toignore_baseline, segpath, msample=minsample,
                                                           varID='Baseline', debug_mode=debug_mode)

    if debug_mode:
        print('DEBUG: 002 Baseline')

    # =============================================================================
    # Code for testing
    #     train_to_split_woe=stratX_train_LimNonLim_ORI
    #     valid_to_split_woe=stratX_test_LimNonLim_ORI
    # =============================================================================

    # Created baseline model on WOE vars
    (baseline_scored_train, baseline_scored_valid, baseline_logreg_model, baseline_logreg_varlist) = ml_logreg(
            train_to_split_woe, valid_to_split_woe, train_var_list, debug_mode=debug_mode, min_Gini_diff=min_Gini_diff, max_vif=max_vif)

    if debug_mode:
        print('DEBUG: 003 Baseline')

    ml_save_train_valid_seg(baseline_scored_train, baseline_scored_valid, None, segpath, segid, 'baseline')

    if debug_mode:
        print('DEBUG: 001')

    # 2) For each variable in varlist that is not in ignore list
    for var in loop_var_list:

        try:

            if debug_mode:
                print('DEBUG: 001a')

            # 2.1) Find a value that splits population in half or creates a min x% subset
            left_subset_train, right_subset_train, split_val = ml_split_train(var, train_to_split, min_group=split_min_group, debug_mode=debug_mode)
            if debug_mode:
                print('DEBUG: 001aa')
            left_subset_valid, right_subset_valid = ml_split_valid(var, valid_to_split, split_val, debug_mode=debug_mode)
            if (left_subset_train is None) | (right_subset_train is None) | (left_subset_valid is None) | (right_subset_valid is None):
                print("NOTE: no minimum split found for var "+var)
                continue

            if debug_mode:
                print('DEBUG: 001b')

            # 2.2) Create dedicated models for each subset by Gini loop
            # Replace raw values with WOE
            left_subset_train_woe,  left_subset_valid_woe  = segmentWOE_ML(segid, left_subset_train,  left_subset_valid,  left_toignore,  segpath, msample=minsample, debug_mode=debug_mode)
            right_subset_train_woe, right_subset_valid_woe = segmentWOE_ML(segid, right_subset_train, right_subset_valid, right_toignore, segpath, msample=minsample, debug_mode=debug_mode)
            # =============================================================================
            # Code for testing
            #             left_subset_train_woe = left_subset_train
            #             left_subset_valid_woe = left_subset_valid
            #             right_subset_train_woe= right_subset_train
            #             right_subset_valid_woe= right_subset_valid
            # =============================================================================

            if debug_mode:
                print('DEBUG: 002')

            # Created models on WOE vars
            left_scored_train,  left_scored_valid,  left_logreg_model,  left_logreg_varlist  = ml_logreg(left_subset_train_woe,  left_subset_valid_woe,  train_var_list, debug_mode=debug_mode, min_Gini_diff=min_Gini_diff, max_vif=max_vif)
            right_scored_train, right_scored_valid, right_logreg_model, right_logreg_varlist = ml_logreg(right_subset_train_woe, right_subset_valid_woe, train_var_list, debug_mode=debug_mode, min_Gini_diff=min_Gini_diff, max_vif=max_vif)

            if debug_mode:
                print('DEBUG: 003')

            # 2.3) Concatenate model scores from both subsets to calculate Gini improvement
            gini_gain_train, gini_gain_valid, train_scored_LR, valid_scored_LR = ml_ginigain(
                    baseline_scored_train, baseline_scored_valid, left_scored_train, left_scored_valid, right_scored_train, right_scored_valid,
                    segpath, segid, var)

            if debug_mode:
                print('DEBUG: 004')

            # 2.4) Store cumulative Gini improvement and data required to plot ROC
            temp=[var, gini_gain_train, gini_gain_valid]
            var_gini_gain.append(temp)
            ml_save_train_valid_seg(train_scored_LR, valid_scored_LR, var_gini_gain, segpath, segid, var)

            if debug_mode:
                print('DEBUG: 005')

        except:
            if debug_mode:
                print('DEBUG: 006')
            temp=[var, np.nan, np.nan]
            var_gini_gain.append(temp)
            ml_save_train_valid_seg(None, None, var_gini_gain, segpath, segid, var)
            continue

    if debug_mode:
        print('DEBUG: 007')

    df_var_gini_gain = pd.DataFrame(var_gini_gain, columns = ["varname","Train Gini Gain","Valid Gini Gain"]).sort_values(by='Valid Gini Gain', ascending='False')

    return df_var_gini_gain


def ml_save_train_valid_seg(train_scored_LR, valid_scored_LR, var_gini_gain, segpath, segid, var):

    if ((train_scored_LR is None) | (valid_scored_LR is None)) == False:
        train_scored_LR['source']='Train'
        valid_scored_LR['source']='Valid'
        TV_scored_LR = pandas.concat([train_scored_LR, valid_scored_LR])
        os.chdir(segpath)
        TV_scored_LR.to_csv('TV_Scored_LR_'+segid+'_'+var+'.csv', index=False)
    else:
        print("WARNING: ml_save_train_valid_seg received empty tables")

    if (var_gini_gain is None) == False:
        pandas.DataFrame(data=var_gini_gain, columns = ["varname","Train Gini Gain","Valid Gini Gain"]).to_csv('VarGiniGain_'+segid+'_accum_loop.csv', index=False)


def ml_split_train(var, train_to_split, min_group = 0.30, debug_mode=False):

# =============================================================================
# Code for testing
# dfCopy = stratX_train_LimNonLim_ORI.copy()
# A,C = ml_split_train('RNILF04', dfCopy)
# B=A['SplitSide'].value_counts(dropna=False)
# A==[]
# dfCopy2 = stratX_train_LimNonLim_ORI.copy()
# dfCopy2['RNILF04'].value_counts(dropna=False)
# dfCopy2['RNILF04'] = np.where( dfCopy2['RNILF04']<-1.0, np.nan, dfCopy2['RNILF04'])
# dfCopy2['RNILF04'].value_counts(dropna=False)
# A,C = ml_split_train('RNILF04', dfCopy2)
# B=A['SplitSide'].value_counts(dropna=False).reset_index()
# train_to_split = dfCopy2
# var = 'RNILF04'
# group_sizes['left'] = (group_sizes['index'])[0].closed
# A=group_sizes.sort_values(by='index', ascending='True')
# =============================================================================

    if debug_mode:
        print('DEBUG ml_split_train: 001')

    split_copy = train_to_split.copy()
    split_copy['SplitSide']=pandas.qcut(train_to_split[var], 2, duplicates='drop')

    if debug_mode:
        print('DEBUG ml_split_train: 002')

    overall_size = split_copy.shape[0]
    group_sizes = split_copy['SplitSide'].value_counts(dropna=False).reset_index()

    if debug_mode:
        print('DEBUG ml_split_train: 003')

    # Allocate NaN to smallest group
    if (group_sizes.loc[group_sizes['index'].isnull()]).shape[0] > 0:
        print("NOTE: ml_split_train "+var+" includes NaN")
        group_to_allocate_nan = ((group_sizes.loc[~group_sizes['index'].isnull()]).sort_values(by='SplitSide', ascending='False').iloc[0])['index']
        #group_sizes['index'] = np.where(group_sizes['index'].isnull(),group_to_allocate_nan,group_sizes['index'])
        split_copy['SplitSide'] = np.where(split_copy['SplitSide'].isnull(),group_to_allocate_nan,split_copy['SplitSide'])
        group_sizes = split_copy['SplitSide'].value_counts(dropna=False).reset_index()

    if debug_mode:
        print('DEBUG ml_split_train: 004')

    group_sizes['ColPct'] = group_sizes['SplitSide']/overall_size

    if debug_mode:
        print('DEBUG ml_split_train: 005')

    if min(group_sizes['ColPct']) < min_group:
        print('ERROR ml_split_train: group_sizes[ColPct]) < min_group for '+var)
        return None, None

    if debug_mode:
        print('DEBUG ml_split_train: 006')

    if group_sizes.shape[0] > 1:
        try:
            #group_sizes=group_sizes.loc[~group_sizes['index'].isnull()]
            group_sizes['left_lim']=0.0
            (group_sizes['left_lim'])[0]=(group_sizes['index'])[0].left
            (group_sizes['left_lim'])[1]=(group_sizes['index'])[1].left
            #(group_sizes['left_lim'])[2]=(group_sizes['index'])[2].left
            group_sizes.sort_values(by='left_lim', inplace=True)
            group_sizes.reset_index(drop=True, inplace=True)
            left_subset_train  = split_copy.loc[split_copy['SplitSide']==(group_sizes['index'])[0]]
            right_subset_train = split_copy.loc[split_copy['SplitSide']==(group_sizes['index'])[1]]
            left_subset_train.drop(['SplitSide'], axis=1, inplace=True)
            right_subset_train.drop(['SplitSide'], axis=1, inplace=True)
            split_val_lowest = (group_sizes['index'])[0].left
            split_val = (group_sizes['index'])[0].right
            split_val_highest = (group_sizes['index'])[1].right

            if ~((split_val_lowest<=split_val) & (split_val<=split_val_highest)):
                print('ERROR: ~((split_val_lowest<=split_val) & (split_val<=split_val_highest))')
                print('split_val_lowest=',split_val_lowest,' split_val=',split_val,' split_val_highest=',split_val_highest)
                return None, None, np.NaN

            if (((group_sizes['index'])[0].closed == 'right') == False):
                print('ERROR: (((group_sizes[\'index\'])[0].closed == \'right\') == False)')
                return None, None, np.NaN

        except:
            print('ERROR: ml_split_train  group_sizes.shape[0] > 1 generated an exception')
            return None, None, np.NaN
    else:
        try:
            left_subset_train  = split_copy.loc[split_copy[var]==1]
            right_subset_train = split_copy.loc[split_copy[var]==2]
            left_subset_train.drop(['SplitSide'], axis=1, inplace=True)
            right_subset_train.drop(['SplitSide'], axis=1, inplace=True)
            split_val=1
        except:
            print('ERROR: ml_split_train has no valid values to split')
            return None, None, np.NaN

    if debug_mode:
        print('DEBUG ml_split_train: 007')

    return left_subset_train, right_subset_train, split_val


def ml_split_valid(var, valid_to_split, split_val, debug_mode=False):

    if debug_mode:
        print('DEBUG ml_split_valid: 001')

    if ((valid_to_split is None) | (split_val == np.NaN) )==True:
        print("WARNING: ml_split_valid called with blanks")
        return None, None

    if debug_mode:
        print('DEBUG ml_split_valid: 002')

    left_subset_valid  = valid_to_split.loc[valid_to_split[var] <= split_val]

    if debug_mode:
        print('DEBUG ml_split_valid: 003')

    right_subset_valid = valid_to_split.loc[valid_to_split[var] >  split_val]

    if debug_mode:
        print('DEBUG ml_split_valid: 004')

    return left_subset_valid, right_subset_valid


import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

def ml_ginigain(baseline_scored_train, baseline_scored_valid, left_scored_train, left_scored_valid, right_scored_train, right_scored_valid, segpath, segid, var):

# =============================================================================
# Code for testing
#     left_scored_train  = pandas.merge(stratX_train_S001[['OpportunityID','target']], scored_train_S001, how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})
#     right_scored_train = pandas.merge(stratX_train_S002[['OpportunityID','target']], scored_train_S002, how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})
#     left_scored_valid  = pandas.merge(stratX_test_S001[['OpportunityID','target']],  scored_test_S001,  how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})
#     right_scored_valid = pandas.merge(stratX_test_S002[['OpportunityID','target']],  scored_test_S002,  how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})
#     baseline_scored_train = pandas.merge(stratX_train_LimNonLim_ORI[['OpportunityID','target']],  logRegTrainScored_LimNonLim_ORI,  how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})
#     baseline_scored_valid = pandas.merge(stratX_test_LimNonLim_ORI[['OpportunityID','target']],   logRegTestScored_LimNonLim_ORI,   how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})
#     segpath=r"U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\4. Commercial model";
#     segid="TESTRUN"
#     var="Thickness"
# =============================================================================

    train_scored_LR = pandas.concat([left_scored_train, right_scored_train])
    valid_scored_LR = pandas.concat([left_scored_valid, right_scored_valid])

     # Plot all models together
    mult=-1.0

    plt.figure(figsize=(8, 6))
    lw = 2

    # Reg Train ORI
    fpr1, tpr1, thresholds1 = metrics.roc_curve(baseline_scored_train['target'], baseline_scored_train['PD']*mult, pos_label=0)
    roc_auc1 = auc(fpr1, tpr1)
    Gini_train_ORI = 2*roc_auc1-1
    Nconsumer = baseline_scored_train.shape[0]
    plt.plot(fpr1, tpr1, ls='dashed', color='xkcd:light purple',
             lw=1, label='Non-segmented Train (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini_train_ORI,Nconsumer))

    # Reg Valid ORI
    fpr1, tpr1, thresholds1 = metrics.roc_curve(baseline_scored_valid['target'], baseline_scored_valid['PD']*mult, pos_label=0)
    roc_auc1 = auc(fpr1, tpr1)
    Gini_valid_ORI = 2*roc_auc1-1
    Nconsumer = baseline_scored_valid.shape[0]
    plt.plot(fpr1, tpr1, ls='solid', color='xkcd:light purple',
             lw=2, label='Non-segmented Valid (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini_valid_ORI,Nconsumer))

    # Reg Train SEG
    fpr1, tpr1, thresholds1 = metrics.roc_curve(train_scored_LR['target'], train_scored_LR['PD']*mult, pos_label=0)
    roc_auc1 = auc(fpr1, tpr1)
    Gini_train_LR = 2*roc_auc1-1
    Nconsumer = train_scored_LR.shape[0]
    plt.plot(fpr1, tpr1, ls='dashed', color='xkcd:blue',
             lw=1, label='Segmented Train (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini_train_LR,Nconsumer))

    # Reg Valid SEG
    fpr1, tpr1, thresholds1 = metrics.roc_curve(valid_scored_LR['target'], valid_scored_LR['PD']*mult, pos_label=0)
    roc_auc1 = auc(fpr1, tpr1)
    Gini_valid_LR = 2*roc_auc1-1
    Nconsumer = valid_scored_LR.shape[0]
    plt.plot(fpr1, tpr1, ls='solid', color='xkcd:blue',
             lw=2, label='Segmented Valid (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini_valid_LR,Nconsumer))

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Megaloop ROC SegID:'+segid+' Var:'+var)
    plt.legend(loc="lower right")

    os.chdir(segpath)
    plt.savefig('ROC_'+segid+'_'+var+'.png')

    gini_gain_train = Gini_train_LR - Gini_train_ORI
    gini_gain_valid = Gini_valid_LR - Gini_valid_ORI

    return gini_gain_train, gini_gain_valid, train_scored_LR, valid_scored_LR

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from segmentWOE import fVarnameGini_ML, fVarnameGini_TT
os.chdir(r'U:\Marketing and Risk\Credit\Auto-decisioning\2017-18 Scorecard build\Bernard\segmentWOE')
from segmentWOE_EG_v2 import fVarnameGini_CV

def ml_logreg(train_df, valid_df, train_var_list, max_var_count=15, max_vif=2.9, min_Gini_diff = 0.003, debug_mode=False, test_predict=True, crossval_predict=False):

# =============================================================================
# Code for testing
#     max_var_count=15
#     max_vif=2.9
#     min_Gini_diff = 0.003
#     debug_mode=False
#     train_df=left_subset_train_woe
#     valid_df=left_subset_valid_woe
#     train_var_list=varlist_reduced
#     train_var_list=varlist_LimNonLim_ORI
# =============================================================================

    varlist = []
    targetvar=['target']

    if debug_mode:
        print('DEBUG: 001')

    # Create initial model on top varnamesGini contributor
    if crossval_predict==False:
        varnamesGini = fVarnameGini_TT(train_df, train_df['target'], valid_df, valid_df['target'], varlist, train_var_list, test_predict)
    else:
        varnamesGini = fVarnameGini_CV(train_df, train_df['target'], varlist, train_var_list, 5)
        varnamesGini['Gini']=varnamesGini['cv_gini']

    if debug_mode:
        print('DEBUG: 001a')

    varlist = [varnamesGini['var'].iloc[0]]

    if debug_mode:
        print('DEBUG: 001b')

    logreg_model, logreg_gini_train, logreg_gini_valid, train_proba, valid_proba = dtreeRegROCWOE_ML(train_df[varlist+targetvar], valid_df[varlist+targetvar])

    if debug_mode:
        print('DEBUG: 002')

    # Add more vars while Gini improvement is significant or we have max variables
    while True:
        # Assess Gini improvement of next var and stop if low improvement
        if crossval_predict==False:
            varnamesGini = fVarnameGini_TT(train_df, train_df['target'], valid_df, valid_df['target'], varlist, train_var_list, test_predict)
        else:
            varnamesGini = fVarnameGini_CV(train_df, train_df['target'], varlist, train_var_list, 5)
            varnamesGini['Gini']=varnamesGini['cv_gini']
        if ((varnamesGini['Gini'].iloc[0] - logreg_gini_valid) < min_Gini_diff):
            break
        candidate_var = [varnamesGini['var'].iloc[0]]
        candidate_varlist = varlist + candidate_var
        vif = [variance_inflation_factor(train_df[candidate_varlist].values, i) for i in range(train_df[candidate_varlist].shape[1])]
        # Do not add new var if vif threshold is breached
        if max(pd.DataFrame(vif)[0]) > max_vif:
            break
        # Create model including added new var
        varlist = candidate_varlist
        (logreg_model, logreg_gini_train, logreg_gini_valid,
         train_proba, valid_proba) = dtreeRegROCWOE_ML(
                 train_df[varlist+targetvar], valid_df[varlist+targetvar])
        if len(varlist)>=max_var_count:
            break

    if debug_mode:
        print('DEBUG: 003')

    # Merge PDs with target values so concatenated Ginis can be computed outside of this function
    scored_train = pandas.merge((train_df.reset_index())[['OpportunityID','target']], train_proba, how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})

# =============================================================================
#     # -- Check that the merge worked properly
#     from sklearn import metrics
#     import matplotlib.pyplot as plt
#     plt.figure()
#     lw = 2
#     fpr1, tpr1, thresholds1 = metrics.roc_curve(scored_train['target'], scored_train['PD']*-1.0, pos_label=0)
#     roc_auc1 = auc(fpr1, tpr1)
#     Gini2 = 2*roc_auc1-1
#     Nconsumer = scored_train.shape[0]
#     plt.plot(fpr1, tpr1, color='blue',
#              lw=1, label='Reg on merged (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini2,Nconsumer))
#     fpr1, tpr1, thresholds1 = metrics.roc_curve(scored_train['target'],
#                                                 logreg_model.predict_proba(train_df[varlist])[:,1]*-1.0, pos_label=0)
#     roc_auc1 = auc(fpr1, tpr1)
#     Gini2 = 2*roc_auc1-1
#     Nconsumer = scored_train.shape[0]
#     plt.plot(fpr1, tpr1, color='green',
#              lw=1, label='Reg on fitter (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini2,Nconsumer))
#     plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Model ROC')
#     plt.legend(loc="lower right")
#     plt.show()
# =============================================================================

    scored_valid = pandas.merge((valid_df.reset_index())[['OpportunityID','target']], valid_proba, how = 'inner', left_index=True, right_index=True).rename(columns={0:'PD'})

# =============================================================================
#     # -- Check that the merge worked properly
#     from sklearn import metrics
#     import matplotlib.pyplot as plt
#     plt.figure()
#     lw = 2
#     fpr1, tpr1, thresholds1 = metrics.roc_curve(scored_valid['target'], scored_valid['PD']*-1.0, pos_label=0)
#     roc_auc1 = auc(fpr1, tpr1)
#     Gini2 = 2*roc_auc1-1
#     Nconsumer = scored_valid.shape[0]
#     plt.plot(fpr1, tpr1, color='blue',
#              lw=1, label='Reg on merged (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini2,Nconsumer))
#     fpr1, tpr1, thresholds1 = metrics.roc_curve(scored_valid['target'],
#                                                 logreg_model.predict_proba(valid_df[varlist])[:,1]*-1.0, pos_label=0)
#     roc_auc1 = auc(fpr1, tpr1)
#     Gini2 = 2*roc_auc1-1
#     Nconsumer = scored_valid.shape[0]
#     plt.plot(fpr1, tpr1, color='green',
#              lw=1, label='Reg on fitter (AUC = %0.2f Gini = %0.2f N=%0.0f)' %(roc_auc1,Gini2,Nconsumer))
#     plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Model ROC')
#     plt.legend(loc="lower right")
#     plt.show()
# =============================================================================

    logreg_varlist = varlist

    return scored_train, scored_valid, logreg_model, logreg_varlist